In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

####1.1. Загрузите агрегированные данные о поездках в мае 2016.

In [0]:
filepath = '/content/gdrive/My Drive/ml/'
filename = '2016-5.csv'
#filename = '2014-12.csv'
data = pd.read_csv(filepath + filename, index_col = 0)

#### 1.2. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки.

In [0]:
#Вытащить массив
data_arr = data.to_numpy()
#len(sum(data_arr)) #2500
data_sum = sum(data_arr)
print 'Quantity of null-cells:', len(data_sum[data_sum == 0])

Quantity of null-cells: 1283


####2.1. Нарисуйте статическую карту Нью-Йорка.
####Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553
####2.2. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.

In [0]:
import folium
long = (-74.25559 - 73.70001)/2
lat  = (40.49612  + 40.91553)/2

#Коррдинаты Empire State Building
latitude_esb  =  40.7484831
longitude_esb = -73.9856715

m = folium.Map(location=[lat,long], zoom_start = 9, no_touch = True)

folium.Marker([latitude_esb , longitude_esb], popup='ESB').add_to(m)

m

/usr/local/lib/python2.7/dist-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


####3. Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.

In [0]:
!pip install geojson

In [0]:
regions = pd.read_csv('/content/gdrive/My Drive/ml/c6/data' + 'regions.csv', ';')

In [0]:
import geojson
from geojson import Feature, Polygon, FeatureCollection

def coord(i):
  return [[(regions.west[i], regions.south[i]), (regions.west[i], regions.north[i]), (regions.east[i], regions.north[i]), (regions.east[i], regions.south[i])]]

qnt = 2500

#make DataFrame
ind_list = [regions.region[i] for i in range(qnt)]
data_qnt = pd.DataFrame(data=ind_list, columns = ['ind'])
data_qnt['qnt'] = pd.Series(data_sum, index=data_qnt.index)

ind_list = [regions.region[i] for i in range(qnt)]

data = FeatureCollection([Feature(geometry = Polygon(coord(i)), id = regions.region[i]) for i in range(qnt)])
#Drow map
m = folium.Map(location=[lat,long], zoom_start = 11, no_touch = True)

folium.Choropleth(
    geo_data=data,
    data=data_qnt,
    columns=['ind', 'qnt'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2    
).add_to(m)

m

####4. Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится статуя свободы.

In [0]:
long = (-74.25559 - 73.70001)/2
lat  = (40.49612  + 40.91553)/2

#Коррдинаты статуи свободы
latitude_sol  =  40.689224
longitude_sol = -74.044547

m = folium.Map(location=[lat,long], zoom_start = 11, no_touch = True)

folium.Marker([latitude_sol , longitude_sol], popup='SOL').add_to(m)

m

####5. Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

In [0]:
data_mean = np.asarray([data_arr.T[i].mean() for i in range(len(data_arr.T))])

qnt = 2500

#make DataFrame
ind_list = [regions.region[i] for i in range(qnt)]
data_m = pd.DataFrame(data=ind_list, columns = ['ind'])
data_m['mean'] = pd.Series(data_mean, index=data_m.index)

ind_list = [regions.region[i] for i in range(qnt)]

data = FeatureCollection([Feature(geometry = Polygon(coord(i)), id = regions.region[i]) for i in range(qnt)])
#Drow map
m = folium.Map(location=[lat,long], zoom_start = 11, no_touch = True)

folium.Choropleth(
    geo_data=data,
    data=data_m,
    columns=['ind', 'mean'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2    
).add_to(m)

m

####6.1. Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся.

In [0]:
data_mean_filtered = data_mean[data_mean >= 5]
print 'Number of cells: ' ,len(data_mean_filtered)

Number of cells:  102


####6.2  Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

In [0]:
# Получить индексы ячеек для удаления
rem_ind = np.asarray([i if (data_mean > 5)[i] == True else 0 for i in range(len(data_mean))])
rem_ind = (rem_ind[rem_ind > 0]).tolist()

data = FeatureCollection([Feature(geometry = Polygon(coord(i)), id = regions.region[i]) for i in rem_ind])
#Drow map
m = folium.Map(location=[lat,long], zoom_start = 11, no_touch = True)

folium.Choropleth(
    geo_data=data
).add_to(m)

m

In [0]:
rem_ind

[1074,
 1075,
 1076,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1178,
 1179,
 1180,
 1181,
 1182,
 1183,
 1220,
 1221,
 1222,
 1223,
 1224,
 1226,
 1227,
 1228,
 1229,
 1230,
 1231,
 1232,
 1233,
 1234,
 1271,
 1272,
 1273,
 1277,
 1278,
 1279,
 1280,
 1281,
 1282,
 1283,
 1284,
 1285,
 1286,
 1325,
 1326,
 1330,
 1331,
 1332,
 1333,
 1334,
 1335,
 1336,
 1337,
 1338,
 1375,
 1376,
 1377,
 1379,
 1381,
 1382,
 1383,
 1384,
 1385,
 1386,
 1387,
 1388,
 1389,
 1425,
 1430,
 1433,
 1434,
 1435,
 1436,
 1437,
 1438,
 1440,
 1441,
 1479,
 1481,
 1482,
 1529,
 1531,
 1532,
 1579,
 1629,
 1683,
 1732,
 1733,
 1782,
 2067,
 2068,
 2117,
 2118,
 2167]

####7. Сохраните ноутбук в формате html, запакуйте ipynb и html в один архив и загрузите его в форму (html облегчит задачу проверки вашим рецензентам — в ipynb динамические карты часто не сохраняются, а для выполнения кода рецензентам придётся устанавливать все библиотеки). 